In [1]:
import os
path=os.getcwd()
print(path)
os.chdir('lib')
from pipeline.forecast import Forecast
import traceback
import time
import datetime
from pipeline.settings import *
from pipeline.secrets import *

import logging
import zipfile
os.chdir(path)   


# Set up logger
logging.root.handlers = []
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.DEBUG, filename='ex.log')
# set up logging to console
console = logging.StreamHandler()
console.setLevel(logging.INFO)
# set a format which is simpler for console use
formatter = logging.Formatter('%(asctime)s : %(levelname)s : %(message)s')
console.setFormatter(formatter)
logging.getLogger("").addHandler(console)


c:\Users\BOttow\OneDrive - Rode Kruis\Documenten\repositories\ibf-rainfall-trigger-kasese\pipeline


In [2]:
startTime = time.time()
logger = logging.getLogger(__name__)
 
logger.info(str(datetime.datetime.now()))
## download data from Datalacke 


2023-04-25 16:22:34,016 : INFO : 2023-04-25 16:22:34.016254


In [3]:
COUNTRY_CODES
SETTINGS_SECRET

{'UGA': {'mock': False, 'notify_email': False}}

In [30]:
COUNTRY_CODE = COUNTRY_CODES[0]
COUNTRY_SETTINGS = SETTINGS[COUNTRY_CODE]
print(COUNTRY_SETTINGS)
LEAD_TIMES = COUNTRY_SETTINGS['lead_times']
print(LEAD_TIMES)

{'lead_times': {'1-day': 1, '2-day': 2, '3-day': 3}, 'thresholds': {'1-day': 60}, 'admin_level': 3, 'EXPOSURE_DATA_SOURCES': {'population': {'source': 'population/hrsl_uga_pop_resized_100', 'rasterValue': 0.15}}}
{'1-day': 1, '2-day': 2, '3-day': 3}


In [32]:
leadTimeLabel = '1-day'
leadTimeValue = 1


In [33]:
logger.info(f'--------STARTING: {leadTimeLabel}' + '--------------------------')
fc = Forecast(leadTimeLabel, leadTimeValue, COUNTRY_CODE,COUNTRY_SETTINGS['admin_level'])
fc.rainfallData.process()
logger.info('--------Finished Rainfall data Processing')

2023-04-25 16:01:45,404 : INFO : --------STARTING: 1-day--------------------------
0it [00:00, ?it/s]

weather predictions for gridpoints...


47it [00:09,  4.76it/s]
0it [00:00, ?it/s]

created: data/other/input/shape/rainfal_long_format.geojson
----------------


save into TIF format....
created: data/raster/output/0/rainfall_extents/rainfall_extent_1-day_UGA.tif
----------------


zonal stats...


66it [00:04, 16.43it/s]


created: data/raster/output/0/rainfall_extents\rainfall_zonal.csv
----------------


determining daily aggregates per catchment area....
daily_rainfall_per_catchment:    hours_ahead  tot_rainfall_mm                 name  trigger
0       1-day             2.51              Bukonjo        0
0       2-day             1.94              Bukonjo        0
0       3-day             3.20              Bukonjo        0
0       1-day             6.35            Busongora        0
0       2-day             3.40            Busongora        0
0       3-day            12.85            Busongora        0
0       1-day             0.00  Kasese Municipality        0
0       2-day             0.00  Kasese Municipality        0
0       3-day             0.00  Kasese Municipality        0
created:data/raster/output/0/rainfall_extents\rainfall_zonal_daily.csv
determining daily aggregates for every location....


2023-04-25 16:02:02,269 : INFO : --------Finished Rainfall data Processing


----------------


done


In [42]:
indicator = 'population'
values = {'source': 'population/hrsl_uga_pop_resized_100', 'rasterValue': 0.15}

In [44]:
print('indicator: ', indicator)
fc.exposure.inputRaster = RASTER_INPUT + values['source'] + ".tif"
fc.exposure.outputRaster = RASTER_OUTPUT + "0/" + \
                values['source'] + fc.exposure.leadTimeLabel

indicator:  population


In [53]:
if os.getenv('TRIGGER') == 'False': # filter only events has high impact
     stats = fc.exposure.calcAffected(fc.exposure.disasterExtentEmptyRaster, indicator, values['rasterValue'])
else:
     stats = fc.exposure.calcAffected(fc.exposure.disasterExtentRaster, indicator, values['rasterValue'])

In [56]:
result = {
                'countryCodeISO3': fc.exposure.countryCodeISO3,
                'exposurePlaceCodes': stats,
                'leadTime': fc.exposure.leadTimeLabel,
                'dynamicIndicator': indicator + '_affected',
                'adminLevel': fc.exposure.admin_level
            }

In [57]:
result

{'countryCodeISO3': 'UGA',
 'exposurePlaceCodes': [{'amount': 0, 'placeCode': 'UG411501'},
  {'amount': 0, 'placeCode': 'UG411502'},
  {'amount': 0, 'placeCode': 'UG411503'}],
 'leadTime': '1-day',
 'dynamicIndicator': 'population_affected',
 'adminLevel': 3}

In [3]:

for COUNTRY_CODE in COUNTRY_CODES:
    logger.info(f'--------STARTING: {COUNTRY_CODE}' + '--------------------------')
    COUNTRY_SETTINGS = SETTINGS[COUNTRY_CODE]
    LEAD_TIMES = COUNTRY_SETTINGS['lead_times']

    for leadTimeLabel, leadTimeValue in LEAD_TIMES.items():
        logger.info(f'--------STARTING: {leadTimeLabel}' + '--------------------------')
        fc = Forecast(leadTimeLabel, leadTimeValue, COUNTRY_CODE,COUNTRY_SETTINGS['admin_level'])
        fc.rainfallData.process()
        logger.info('--------Finished Rainfall data Processing')
        fc.exposure.callAllExposure()
        logger.info('--------Finished exposure')
        if COUNTRY_CODE =='SSD':
            fc.exposure.makeMaps()
            logger.info('--------Finished make maps')                
        fc.db.upload()                
        logger.info('--------Finished upload')
        fc.db.sendNotification()
        logger.info('--------Finished notification')
 
    elapsedTime = str(time.time() - startTime)
    logger.info(str(elapsedTime))


2023-04-25 16:22:46,221 : INFO : --------STARTING: UGA--------------------------
2023-04-25 16:22:46,222 : INFO : --------STARTING: 1-day--------------------------
5it [00:00, 45.06it/s]

weather predictions for gridpoints...


47it [00:01, 39.76it/s]


created: data/other/input/shape/rainfal_long_format.geojson
----------------


save into TIF format....


0it [00:00, ?it/s]

...

Uploaded alert_threshold
Uploaded raster-file: data/raster/output/0/rainfall_extents/rainfall_extent_2-day_UGA.tif


2023-04-25 16:23:00,980 : INFO : --------Finished Rainfall data Processing
2023-04-25 16:23:01,060 : INFO : --------Finished exposure


indicator:  population
result:  {'countryCodeISO3': 'UGA', 'exposurePlaceCodes': [{'amount': 0, 'placeCode': 'UG411501'}, {'amount': 0, 'placeCode': 'UG411502'}, {'amount': 0, 'placeCode': 'UG411503'}], 'leadTime': '3-day', 'dynamicIndicator': 'population_affected', 'adminLevel': 3}
self.statsPath:  data/other/output/calculated_affected/affected_3-day_UGA_population.json
indicator:  population
body:  {'countryCodeISO3': 'UGA', 'exposurePlaceCodes': [{'amount': 0, 'placeCode': 'UG411501'}, {'amount': 0, 'placeCode': 'UG411502'}, {'amount': 0, 'placeCode': 'UG411503'}], 'leadTime': '3-day', 'dynamicIndicator': 'population_affected', 'adminLevel': 3, 'disasterType': 'heavy-rain'}
Uploaded calculated_affected for indicator: population
indicator:  alert_threshold
body:  {'countryCodeISO3': 'UGA', 'exposurePlaceCodes': [{'amount': 0, 'placeCode': 'UG411501'}, {'amount': 0, 'placeCode': 'UG411502'}, {'amount': 0, 'placeCode': 'UG411503'}], 'leadTime': '3-day', 'dynamicIndicator': 'alert_thres

2023-04-25 16:23:01,697 : INFO : --------Finished upload
2023-04-25 16:23:01,697 : INFO : --------Finished notification
2023-04-25 16:23:01,698 : INFO : 27.681947231292725


Uploaded alert_threshold
Uploaded raster-file: data/raster/output/0/rainfall_extents/rainfall_extent_3-day_UGA.tif


In [ ]:
import PyPDF2 
import re


In [ ]:
os.getcwd()

In [ ]:

# Open the PDF file
pdf_file = open('../../temp/evidence.pdf', 'rb')
pdf_reader = PyPDF2.PdfFileReader(pdf_file)
# Extract the text from each page of the PDF
text = ''
for page_num in range(pdf_reader.getNumPages()):
    page = pdf_reader.getPage(page_num)
    text += page.extractText()
    print(page_num)
# Use regular expressions to find keywords in the text
keywords = re.findall(r'\b(\w+)\b', text)

# Print the keywords
print(keywords)
pdf_file.close()

In [ ]:
from zotero2readwise.zt2rw import Zotero2Readwise

zt_rw = Zotero2Readwise(
    readwise_token="2y8JuTpQzigk8zQx3IhrXlMAwPM5XI15fki7G98AYCWF1NgFkC",  # Visit https://readwise.io/access_token)
    zotero_key="UL310hGAoZcASmgobT4mPrHb",  # Visit https://www.zotero.org/settings/keys
    zotero_library_id="akliludin", # Visit https://www.zotero.org/settings/keys
    zotero_library_type="user", # "user" (default) or "group"
    include_annotations=True, # Include Zotero annotations -> Default: True
    include_notes=False, # Include Zotero notes -> Default: False
)
zt_rw.run()

In [ ]:
IBF_URL='https://ibf-test.510.global/api/' 
ADMIN_PASSWORD='password' 
#IBF_PASSWORD='rp82pU8Qge6Yvu'
# API-service access
ADMIN_LOGIN = 'dunant@redcross.nl'
import requests
API_LOGIN_URL=IBF_URL+'user/login'



In [ ]:
import pandas as pd
import numpy as np
import json

filename=fc.db.triggerFolder +'triggers_rp_' + fc.db.leadTimeLabel + '_' + fc.db.countryCodeISO3 + ".json"

df = pd.read_json(filename,orient='records')
 
 
with open(filename) as json_file:
    triggers = json.load(json_file) 

 
stationForecasts = []
for key in triggers:
    stationForecasts.append({
        "stationCode": key['stationCode'],
        "forecastLevel": int(key['fc']),
        "eapAlertClass": key['eapAlertClass'],
        "forecastReturnPeriod": key['fc_rp'],
        "triggerLevel": int(key['triggerLevel'])

    })
    
body = {
    "countryCodeISO3": fc.db.countryCodeISO3,
    "leadTime": fc.db.leadTimeLabel,
    "date": fc.db.uploadTime,
    "stationForecasts": stationForecasts
}
    
    
    


In [ ]:
body

In [ ]:
fc.db.apiPostRequest('glofas-stations/triggers', body=body)
logger.info('Uploaded triggers per station')

In [ ]:
    
    dfStation['stationCode'] = df['stationCode']
dfStation['forecastLevel'] = df['fc'].astype(np.int32,errors='ignore')
#dfStation['forecastProbability'] = df['fc_prob'].astype(np.float64,errors='ignore')
#dfStation['forecastTrigger'] = df['fc_trigger'].astype(np.int32,errors='ignore')
dfStation['eapAlertClass'] = df['eapAlertClass']
dfStation['forecastReturnPeriod'] = df['fc_rp'].astype(np.int32,errors='ignore')
dfStation['triggerLevel'] = df['triggerLevel'].astype(np.int32,errors='ignore')


body = {
    'countryCodeISO3': self.countryCodeISO3,
    'triggersPerLeadTime': triggersPerLeadTime
}
body['disasterType'] = self.getDisasterType()
body['date']=self.uploadTime

In [ ]:
df=df.replace(np.nan,null)
dfStation = pd.DataFrame(index=df.index)
dfStation['stationCode'] = df['stationCode']
dfStation['forecastLevel'] = df['fc'].astype(np.int32,errors='ignore')
#dfStation['forecastProbability'] = df['fc_prob'].astype(np.float64,errors='ignore')
#dfStation['forecastTrigger'] = df['fc_trigger'].astype(np.int32,errors='ignore')
dfStation['eapAlertClass'] = df['eapAlertClass']
dfStation['forecastReturnPeriod'] = df['fc_rp'].astype(np.int32,errors='ignore')
dfStation['triggerLevel'] = df['triggerLevel'].astype(np.int32,errors='ignore')
stationForecasts = json.loads(dfStation.to_json(orient='records'))
body = {
    'countryCodeISO3': fc.db.countryCodeISO3,
    'leadTime': fc.db.leadTimeLabel,
    'date': fc.db.uploadTime,
    'stationForecasts': stationForecasts
}
#body['disasterType'] = fc.db.getDisasterType()
#body['date']=fc.db.uploadTime


In [ ]:

fc.db.apiPostRequest('glofas-stations/triggers', body=body)
logger.info('Uploaded triggers per station')

 

In [ ]:
body

In [ ]:

fc.db.apiPostRequest('glofas-stations/triggers', body=body)
logger.info('Uploaded triggers per station')

 

In [ ]:
dffinal = pd.concat(dfs)

In [ ]:
df3=dffinal.groupby('storm_id').agg(DMG_predicted_total=('DMG_predicted','sum')).reset_index()
df3.DMG_predicted_total.values

In [ ]:
np.mean(dffinal.DMG_predicted_t.values)

In [ ]:


Yauyos=dffinal['DMG_predicted_t']
 
Yauyos[Yauyos>0].hist(bins=100, edgecolor='#4aaaaa', color='#80BCD8')
import matplotlib.pyplot as plt

plt.plot(tabulaciones,distnormal, color='#4B4C4E', linewidth=5, linestyle='--',label='Dist Normal')
plt.plot(tabulaciones,distlognormal, color='#3F83B7', linewidth=5, linestyle='--', label='Dist Lognormal')
plt.plot(tabulaciones,distweibull, color='#7B7C7E', linewidth=5, linestyle='-.', label='Dist Weibull')
plt.plot(tabulaciones,distchi2, color='#3F83B7', linewidth=5, linestyle=':', label='Dis Chi2')
#plt.xlim(0,30)
plt.legend(loc='upper right')


In [ ]:
import numpy as np
from scipy import stats
from scipy.stats import genextreme
Yauyos=dffinal.DMG_predicted_t.values
#hallamos el promedio y la desviación
promedio = Yauyos[Yauyos>0].mean()
desviacion = Yauyos[Yauyos>0].std()
print (promedio, desviacion)

#determinamos las regresiones estadisticas
tabulaciones = np.arange(-40,51,0.1)
distnormal = stats.norm.pdf(tabulaciones, 
                            loc=promedio, scale=desviacion)
distlognormal = stats.pearson3.pdf(tabulaciones,skew=1,
                                   loc=promedio, scale=desviacion)
distweibull = stats.dweibull.pdf(tabulaciones,c=1,
                                   loc=promedio, scale=desviacion)
distchi2 = stats.chi2.pdf(tabulaciones,df=2,
                                   loc=promedio, scale=desviacion)

p100y_norm = stats.norm.ppf([0.99], loc=promedio, scale=desviacion)
p100y_lognorm = stats.pearson3.ppf([0.99], skew=1, loc=promedio, scale=desviacion)


for values in[0.5,0.8,0.857142857,0.9,0.933333333,0.95,0.96,0.966666667,0.975,0.98,0.983333333,0.9875,0.99,0.993333333,0.995]:
    year=int((1/(1-values)))
    p100y_lognorm = stats.pearson3.ppf([values], skew=1, loc=promedio, scale=desviacion)
    print (f"damage for {year} return period Dist Logormal =", int(p100y_lognorm[0]))



In [ ]:

data = df3.DMG_predicted_total.values
# Fit the generalized extreme value distribution to the data.
shape, loc, scale = genextreme.fit(data)
print("Fit parameters:")
print(f"  shape: {shape:.4f}")
print(f"  loc:   {loc:.4f}")
print(f"  scale: {scale:.4f}")
print()

# Compute the return levels for several return periods.
return_periods = np.array([5, 10, 20, 50, 100])
return_levels = genextreme.isf(1/return_periods, shape, loc, scale)

print("Return levels:")
print()
print("Period    Level")
print("(years)   (temp)")

for period, level in zip(return_periods, return_levels):
    print(f'{period:4.0f}  {level:9.2f}')